# ATDS Project P4 code (except GNN, LLN, and Data Visualization)


Team Members:
Aadithya kandeth - 6980-2791
Anol Kurian V - 56268544

In [ ]:
# Project Introduction and Instructions

# Project Title: Predictive Analysis of Global Societal Unrest
# Description:
# This project aims to predict and analyze global societal unrest events (like protests, attacks, 
# and arrests) using advanced machine learning models. The primary data sources include the 
# Global Database of Events, Language, and Tone (GDELT) and the Armed Conflict Location & 
# Event Data Project (ACLED). The project employs models like XGBoost, Graph Neural Networks (GNN), 
# and Large Language Models (LLM) to analyze patterns and forecast potential unrest events.

# Instructions:

# 1. Data Preprocessing:
#    - Load and combine data from GDELT and ACLED.
#    - Clean the data and split it into training, validation, and test sets.

# 2. Model Training:
#    - Train the model on the preprocessed data.

# 3. Data Visualization and Analysis:
#    - Visualize the data before and after model training to understand patterns.
#    - Analyze the results from each model for insights and accuracy.

# 4. Integration and Output:
#    - Combine the results from all models.
#    - Generate comprehensive insights and visualizations for decision-making.

# Note: Ensure all necessary libraries and frameworks (like xgboost, torch, dgl, openai) are installed.


## Data Preprocessing

In [1]:
import pandas as pd


df1 = pd.read_csv("./Data/GDELT_merged_1998to2023.csv", sep='\t')
column_names = [
    'GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate',
    'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
    'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
    'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
    'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
    'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
    'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
    'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass',
    'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
    'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
    'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long',
    'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName',
    'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat',
    'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type',
    'ActionGeo_FullName', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code',
    'ActionGeo_Lat', 'ActionGeo_Long', 'ActionGeo_FeatureID',
    'DATEADDED', 'SOURCEURL'
]

df1.columns = column_names
columns_to_drop = ['SOURCEURL', 'DATEADDED','GLOBALEVENTID','FractionDate','Actor1Name', 'Actor1Religion2Code', 'Actor1Type2Code', 'Actor1Type3Code'
                  ,'Actor2Name', 'Actor2Religion2Code', 'Actor2Type2Code', 'Actor2Type3Code','Actor1Geo_FullName','Actor1Geo_ADM1Code','Actor1Geo_FeatureID',
                  'Actor2Geo_FullName','Actor2Geo_ADM1Code','Actor2Geo_FeatureID','ActionGeo_FullName','ActionGeo_ADM1Code','ActionGeo_FeatureID']
df1 = df1.drop(columns=columns_to_drop)

df1['importance'] = (df1['NumMentions'] + df1['NumSources'] + df1['NumArticles']) / 3


def get_top_100_entries(group):
    return group.nlargest(100, 'importance')
df1['SQLDATE'] = pd.to_datetime(df1['SQLDATE'], format='%Y%m%d', errors='coerce')
top_100_per_day = df1.groupby('SQLDATE', group_keys=False, sort=True).apply(get_top_100_entries)


print(df1.shape)
print(df1.columns)

C:\Users\aadit\AppData\Local\Temp\ipykernel_16772\3443576277.py:4: DtypeWarning: Columns (24,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("C:/gdelt/gdelt_data_combined/GDELT_merged_1998to2023.csv", sep='\t')


(2857546, 38)
Index(['SQLDATE', 'MonthYear', 'Year', 'Actor1Code', 'Actor1CountryCode',
       'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code',
       'Actor1Type1Code', 'Actor2Code', 'Actor2CountryCode',
       'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code',
       'Actor2Type1Code', 'IsRootEvent', 'EventCode', 'EventBaseCode',
       'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions',
       'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type',
       'Actor1Geo_CountryCode', 'Actor1Geo_Lat', 'Actor1Geo_Long',
       'Actor2Geo_Type', 'Actor2Geo_CountryCode', 'Actor2Geo_Lat',
       'Actor2Geo_Long', 'ActionGeo_Type', 'ActionGeo_CountryCode',
       'ActionGeo_Lat', 'ActionGeo_Long', 'importance'],
      dtype='object')


In [2]:
# to be removed
filtered_df1 = top_100_per_day[top_100_per_day["MonthYear"] == 202201]



filtered_df1['scaled_mood'] =  (filtered_df1["GoldsteinScale"]* filtered_df1["importance"] + filtered_df1["AvgTone"]* filtered_df1["importance"])/2
columns_to_drop = ['MonthYear','Year','GoldsteinScale','NumMentions','NumSources', 'NumArticles','AvgTone','Actor1Geo_Lat','Actor1Geo_Long',
                  'Actor2Geo_Lat','Actor2Geo_Long','ActionGeo_Lat','ActionGeo_Long', 'Actor1Code','Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code',
                  'Actor2Code','Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor1Geo_Type', 'Actor2Geo_Type']
filtered_df1 = filtered_df1.drop(columns=columns_to_drop)


print(filtered_df1.shape)
print(filtered_df1.head().to_string(index=False))


(3000, 16)
   SQLDATE Actor1CountryCode Actor1Type1Code Actor2CountryCode Actor2Type1Code  IsRootEvent EventCode EventBaseCode EventRootCode  QuadClass Actor1Geo_CountryCode Actor2Geo_CountryCode  ActionGeo_Type ActionGeo_CountryCode  importance  scaled_mood
2022-01-01               NaN             COP               NaN             NaN            1        10            10             1          1                   NaN                   NaN               0                   NaN 2061.666667 -5060.735897
2022-01-01               USA             MNC               NaN             NaN            1       160           160            16          4                    US                   NaN               2                    US 1262.333333 -4852.576083
2022-01-01               NaN             COP               NaN             NaN            1       114           114            11          3                   NaN                   NaN               0                   NaN 1159.666667 -3814.6317

C:\Users\aadit\AppData\Local\Temp\ipykernel_16772\1849667737.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['scaled_mood'] =  (filtered_df1["GoldsteinScale"]* filtered_df1["importance"] + filtered_df1["AvgTone"]* filtered_df1["importance"])/2


In [3]:
import os
import sys
import pandas as pd


df2 = pd.read_json("./Data/acled_all_data.json")
print(df2.columns)
# print(df2.head().to_string(index=False))

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp'],
      dtype='object')


In [4]:
date_country_map = filtered_df1.groupby('SQLDATE')['ActionGeo_CountryCode'].agg(set).to_dict()
print(date_country_map)

{Timestamp('2022-01-01 00:00:00'): {'BR', 'IN', 'RS', 'CH', 'US', 'SF', 'PL', 'AS', 'FR', nan, 'KN', 'RO', 'UK', 'AE', 'UP', 'IT'}, Timestamp('2022-01-02 00:00:00'): {'SU', 'GZ', 'US', 'SF', 'EI', 'AS', 'KS', nan, 'KN', 'UK', 'IS'}, Timestamp('2022-01-03 00:00:00'): {'BR', 'SN', 'SU', 'RS', 'US', 'KS', 'AS', nan, 'KN', 'IR', 'GM', 'UK', 'IS', 'IZ', 'SZ'}, Timestamp('2022-01-04 00:00:00'): {'IZ', 'IC', 'FR', 'CH', 'US', 'SF', 'CB', nan, 'UK', 'JA'}, Timestamp('2022-01-05 00:00:00'): {'LT', 'VE', 'FR', 'HK', 'US', 'SF', 'CH', 'AS', nan, 'KN', 'UK', 'IS', 'KZ'}, Timestamp('2022-01-06 00:00:00'): {'RE', 'PK', 'RS', 'CH', 'US', nan, 'GM', 'BF', 'UK', 'IS', 'KZ'}, Timestamp('2022-01-07 00:00:00'): {'RS', 'FR', 'US', 'CB', 'MX', nan, 'GM', 'UK', 'KZ'}, Timestamp('2022-01-08 00:00:00'): {'AL', 'RS', 'CH', 'US', 'CB', 'FR', 'TH', 'AS', nan, 'ET', 'UK', 'ER', 'UP', 'KZ'}, Timestamp('2022-01-09 00:00:00'): {'BR', 'VT', 'SP', 'SU', 'RS', 'US', nan, 'UK', 'PE', 'CY', 'UP', 'KZ', 'OD'}, Timestamp('2

In [5]:
country_map={'4':'AF','8':'AL','12':'AG','16':'AQ','20':'AN','24':'AO','660':'AV','28':'AC','32':'AR','51':'AM','533':'AA','36':'AS','40':'AU','31':'AJ','44':'BF','48':'BA','50':'BG','52':'BB','112':'BO','56':'BE','84':'BZ','204':'BN','60':'BM','64':'BT','68':'BL','70':'BK','72':'BC','76':'BR','92':'VI','96':'BX','100':'BU','854':'UV','116':'CB','120':'CM','124':'CA','132':'CV','136':'CJ','140':'CT','148':'CD','152':'BL','156':'CH','162':'KT','166':'CK','170':'CO','174':'CN','178':'CF','184':'CW','188':'CS','384':'CI','191':'HR','192':'CU','196':'CY','203':'EZ','208':'DA','262':'DJ','212':'DO','214':'DR','218':'EC','818':'EG','222':'ES','226':'EK','232':'ER','233':'EN','231':'ET','238':'FA','234':'FO','242':'FJ','246':'FI','250':'FR','254':'FG','258':'FP','266':'GB','270':'GA','268':'GE','276':'GM','288':'GH','292':'GI','300':'GR','304':'GL','308':'GJ','312':'GP','316':'GQ','320':'GT','316':'GU','324':'GN','624':'PU','328':'GY','332':'HA','336':'VT','340':'HO','348':'HU','352':'IC','356':'IN','360':'ID','364':'IR','368':'IZ','372':'EI','376':'IS','380':'IT','388':'JM','392':'JA','400':'JO','398':'KZ','404':'KE','408':'KR','410':'KS','414':'KU','417':'KG','418':'LA','428':'LG','422':'LE','430':'LT','438':'LU','440':'LV','442':'LT','807':'MK','450':'MA','454':'MI','458':'MY','462':'MV','466':'ML','470':'MT','474':'MB','478':'MR','480':'MP','175':'YT','484':'MX','583':'FM','498':'MD','492':'MC','500':'MS','504':'MO','508':'MZ','104':'BM','516':'WA','520':'NR','524':'NP','528':'NL','530':'NT','540':'NC','554':'NZ','558':'NU','566':'NG','562':'NE','566':'NI','570':'NU','574':'NF','580':'CQ','578':'NO','512':'MU','586':'PK','585':'PS','591':'PM','598':'PP','600':'PA','604':'PE','608':'RP','612':'PC','616':'PL','620':'PO','630':'RQ','634':'QA','638':'RE','642':'RO','643':'RS','646':'RW','659':'SC','662':'ST','666':'SB','670':'VC','674':'SM','678':'TP','682':'SA','686':'SG','688':'SR','703':'SV','705':'SI','090':'BP','706':'SO','710':'SF','724':'SP','144':'CE','729':'SU','740':'NS','744':'SV','748':'WZ','752':'SW','756':'SZ','760':'SY','158':'TW','762':'TI','834':'TZ','764':'TH','768':'TO','772':'TL','776':'TN','780':'TD','788':'TS','792':'TU','795':'TX','796':'TK','798':'TV','800':'UG','804':'UP','784':'TC','826':'UK','840':'US','858':'UY','860':'UZ','548':'NH','862':'VE','704':'VM','850':'VQ','876':'WF','732':'WI','882':'WS','887':'YM','710':'ZA','894':'ZR','716':'ZI','434':'LY','275':'WE','180':'CG','499':'MJ','728':'OD','0':'XK','108':'BI','663':'MF','496':'MN','831':'GG','694':'SL','296':'KI','626':'TT','531':'CW','426':'LT'}
reverse_country_map = {
    'AF': '4',
    'AL': '8',
    'AG': '12',
    'AQ': '16',
    'AN': '20',
    'AO': '24',
    'AV': '660',
    'AC': '28',
    'AR': '32',
    'AM': '51',
    'AA': '533',
    'AS': '36',
    'AU': '40',
    'AJ': '31',
    'BF': '44',
    'BA': '48',
    'BG': '50',
    'BB': '52',
    'BO': '112',
    'BE': '56',
    'BZ': '84',
    'BN': '204',
    'BM': '60',
    'BT': '64',
    'BL': '68',
    'BK': '70',
    'BC': '72',
    'BR': '76',
    'VI': '92',
    'BX': '96',
    'BU': '100',
    'UV': '854',
    'CB': '116',
    'CM': '120',
    'CA': '124',
    'CV': '132',
    'CJ': '136',
    'CT': '140',
    'CD': '148',
    'BL': '152',
    'CH': '156',
    'KT': '162',
    'CK': '166',
    'CO': '170',
    'CN': '174',
    'CF': '178',
    'CW': '184',
    'CS': '188',
    'CI': '384',
    'HR': '191',
    'CU': '192',
    'CY': '196',
    'EZ': '203',
    'DA': '208',
    'DJ': '262',
    'DO': '212',
    'DR': '214',
    'EC': '218',
    'EG': '818',
    'ES': '222',
    'EK': '226',
    'ER': '232',
    'EN': '233',
    'ET': '231',
    'FA': '238',
    'FO': '234',
    'FJ': '242',
    'FI': '246',
    'FR': '250',
    'FG': '254',
    'FP': '258',
    'GB': '266',
    'GA': '270',
    'GE': '268',
    'GM': '276',
    'GH': '288',
    'GI': '292',
    'GR': '300',
    'GL': '304',
    'GJ': '308',
    'GP': '312',
    'GQ': '316',
    'GT': '320',
    'GU': '316',
    'GN': '324',
    'PU': '624',
    'GY': '328',
    'HA': '332',
    'VT': '336',
    'HO': '340',
    'HU': '348',
    'IC': '352',
    'IN': '356',
    'ID': '360',
    'IR': '364',
    'IZ': '368',
    'EI': '372',
    'IS': '376',
    'IT': '380',
    'JM': '388',
    'JA': '392',
    'JO': '400',
    'KZ': '398',
    'KE': '404',
    'KR': '408',
    'KS': '410',
    'KU': '414',
    'KG': '417',
    'LA': '418',
    'LG': '428',
    'LE': '422',
    'LT': '430',
    'LU': '438',
    'LV': '440',
    'LT': '442',
    'MK': '807',
    'MA': '450',
    'MI': '454',
    'MY': '458',
    'MV': '462',
    'ML': '466',
    'MT': '470',
    'MB': '474',
    'MR': '478',
    'MP': '480',
    'YT': '175',
    'MX': '484',
    'FM': '583',
    'MD': '498',
    'MC': '492',
    'MS': '500',
    'MO': '504',
    'MZ': '508',
    'BM': '104',
    'WA': '516',
    'NR': '520',
    'NP': '524',
    'NL': '528',
    'NT': '530',
    'NC': '540',
    'NZ': '554',
    'NU': '558',
    'NG': '566',
    'NE': '562',
    'NI': '566',
    'NU': '570',
    'NF': '574',
    'CQ': '580',
    'NO': '578',
    'MU': '512',
    'PK': '586',
    'PS': '585',
    'PM': '591',
    'PP': '598',
    'PA': '600',
    'PE': '604',
    'RP': '608',
    'PC': '612',
    'PL': '616',
    'PO': '620',
    'RQ': '630',
    'QA': '634',
    'RE': '638',
    'RO': '642',
    'RS': '643',
    'RW': '646',
    'SC': '659',
    'ST': '662',
    'SB': '666',
    'VC': '670',
    'SM': '674',
    'TP': '678',
    'SA': '682',
    'SG': '686',
    'SR': '688',
    'SV': '703',
    'SI': '705',
    'BP': '090',
    'SO': '706',
    'SF': '710',
    'SP': '724',
    'CE': '144',
    'SU': '729',
    'NS': '740',
    'SV': '744',
    'WZ': '748',
    'SW': '752',
    'SZ': '756',
    'SY': '760',
    'TW': '158',
    'TI': '762',
    'TZ': '834',
    'TH': '764',
    'TO': '768',
    'TL': '772',
    'TN': '776',
    'TD': '780',
    'TS': '788',
    'TU': '792',
    'TX': '795',
    'TK': '796',
    'TV': '798',
    'UG': '800',
    'UP': '804',
    'TC': '784',
    'UK': '826',
    'US': '840',
    'UY': '858',
    'UZ': '860',
    'NH': '548',
    'VE': '862',
    'VM': '704',
    'VQ': '850',
    'WF': '876',
    'WI': '732',
    'WS': '882',
    'YM': '887',
    'ZA': '710',
    'ZR': '894',
    'ZI': '716',
    'LY': '434',
    'WE': '275',
    'CG': '180',
    'MJ': '499',
    'OD': '728',
    'XK': '0',
    'BI': '108',
    'MF': '663',
    'MN': '496',
    'GG': '831',
    'SL': '694',
    'KI': '296',
    'TT': '626',
    'CW': '531',
    'LT': '426'
}

In [6]:
event_index = {
    "Government regains territory": "0",
    "Non-state actor overtakes territory": "1",
    "Armed clash": "2",
    "Excessive force against protesters": "3",
    "Protest with intervention": "4",
    "Peaceful protest": '5',
    "Violent demonstration": "6",
    "Mob violence": "7",
    "Chemical weapon": "8",
    "Air/drone strike": "9",
    "Suicide bomb": "10",
    "Shelling/artillery/missile attack": "11",
    "Remote explosive/landmine/IED": "12",
    "Grenade": "13",
    "Sexual violence": "14",
    "Attack": "15",
    "Abduction/forced disappearance": "16",
    "Agreement": "17",
    "Arrests": "18",
    "Change to group/activity": "19",
    "Disrupted weapons use": "20",
    "Headquarters or base established": "21",
    "Looting/property destruction": "22",
    "Non-violent transfer of territory": "23",
    "Other": "24"
}


In [7]:
xydf1 = filtered_df1

for i in range(25):
    column_name = f'y{i}'  # Generate column name (y0, y1, ..., y24)
    xydf1[column_name] = 0

In [8]:
df2['event_date'] = pd.to_datetime(df2['event_date'])

In [9]:
xydf1_temp = xydf1

for index, row in df2.iterrows():
    date = row['event_date']
    country = row['iso']
    print(row['country'])
    country_fips = country_map[str(country)] 
    sub_event_type = row['sub_event_type']
    column = 'y'+ str(event_index[sub_event_type])
    
    # Check if the associated country is in the hashmap value for the date key
    if country_fips in date_country_map.get(date, []):
        # If it's there, print the row
        
        filter1 = xydf1['SQLDATE'] == date
        filter2 = xydf1['ActionGeo_CountryCode'] == country_fips
        filter3 = xydf1['scaled_mood'] < 0

        xydf1_temp.loc[filter1 & filter2 & filter3, column] = 1
        
        #print(row)

Burkina Faso
Burkina Faso
Burkina Faso
Burkina Faso
Ghana
Mali
Mali
Nigeria
Nigeria
Nigeria
Tunisia
Ivory Coast
Niger
Nigeria
Central African Republic
Nigeria
Nigeria
Morocco
Algeria
Algeria
Libya
Nigeria
Nigeria
Burkina Faso
Burkina Faso
Iran
Israel
Palestine
Turkey
Turkey
Turkey
Israel
Iran
Lebanon
Palestine
Turkey
Costa Rica
Haiti
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Cuba
Cuba
Dominican Republic
Mexico
Puerto Rico
Puerto Rico
El Salvador
El Salvador
El Salvador
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
El Salvador
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Trinidad and Tobago
Mexico
Mexico
Sudan
Tanzania
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
eSwatini
Iraq
Syria
Syria
Syria
Syria
Syria
Syria
Iraq
Iraq
Iraq
Syria
Syria
Syria
Iraq
Syria
Syria
Argentina
Colombia
Brazil
Brazil
Brazil
Brazil
Brazil
Venezuela
Venezuela
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Argentina
Peru
Bolivi

United Kingdom
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
United States
United States
United States
United States
Canada
Canada
Saint Pierre and Miquelon
Canada
Canada
United States
Canada
Canada
Canada
Canada
Canada
United States
Canada
United States
United States
Canada
Canada
United States
Canada
Canada
United States
France
Spain
Netherlands
Myanmar
United States
United States
United States
Belgium
Ukraine
Ukraine
Ukraine
Benin
Nigeria
Nigeria
Nigeria
Nigeria
Nigeria
Nigeria
Nigeria
Iraq
Iraq
Iraq
Iraq
Iraq
Brazil
Brazil
Afghanistan
Brazil
India
Myanmar
Myanmar
Nigeria
Syria
Syria
France
Myanmar
Germany
United Kingdom
United Kingdom
United Kingdom
United Kingdom
United Kingdom
Ethiopia
Ethiopia
Ethiopia
Italy
Yemen
Yemen
Yemen
Yemen
Yemen
Democratic Republic of Congo
Thailand
Bahrain
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Niger
Niger
Germany
Yemen
Yemen
Burkina Faso
Yemen
Yemen
France
Italy
Democratic Republic of Congo
Yemen
South Afr

United States
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
Canada
South Korea
New Zealand
Canada
United States
Myanmar
Italy
Canada
North Korea
Canada
United States
United States
United States
United States
El Salvador
El Salvador
Argentina
Poland
Afghanistan
Cambodia
Central African Republic
Nigeria
Nigeria
Nigeria
Nigeria
India
India
Mexico
Mexico
Myanmar
Denmark
Norway
Afghanistan
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Ukraine
Thailand
Bahrain
Colombia
Colombia
Colombia
Colombia
Niger
Libya
Yemen
Yemen
Yemen
Democratic Republic of Congo
Yemen
Indonesia
Indonesia
Myanmar
France
France
Spain
France
France
France
Yemen
Burkina Faso
Burkina Faso
Burkina Faso
Morocco
Morocco
Morocco
Nigeria
Nigeria
Senegal
Burkina Faso
Tunisia
Tunisia
Chad
Morocco
Nigeria
Burkina Faso
Egypt
Nigeria
Morocco
Morocco
Morocco
Burundi
Cameroon
Nigeria
Nigeria
Senegal
Burkina Faso
Burkina Faso
Mali
Mali
Iran
Palestine
Palestine
Turkey
Turkey
Turkey
Lebanon
Is

Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Serbia
Montenegro
Georgia
Kyrgyzstan
Georgia
Montenegro
Montenegro
Montenegro
Montenegro
Georgia
Kyrgyzstan
Poland
Poland
Montenegro
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Portugal
France
Italy
Italy
United Kingdom
Spain
Spain
Spain
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Austria
Germany
Germany
Germany
Germany
Germany
Germany
Germany
France
France
France
Germany
Germany
Germany
Germany
Germany
United Kingdom
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Germany
Ge

United States
Honduras
Colombia
Mali
eSwatini
Democratic Republic of Congo
Libya
Cambodia
Nigeria
Nigeria
Nigeria
Nigeria
Nigeria
Nigeria
Brazil
Colombia
Guadeloupe
India
Madagascar
Mexico
Somalia
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Ecuador
Bahrain
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Niger
Yemen
Myanmar
Yemen
Yemen
Democratic Republic of Congo
Sudan
Saudi Arabia
Palestine
Palestine
Kenya
Yemen
United States
United States
Cameroon
Cameroon
Ghana
Mali
Nigeria
Nigeria
Nigeria
Algeria
Libya
Nigeria
Nigeria
Senegal
Ghana
Nigeria
Mali
Senegal
Nigeria
Mali
Burkina Faso
Cameroon
Palestine
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Israel
Israel
Iran
Lebanon
Lebanon
Lebanon
Iran
Iran
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Democratic Republic of Congo
Malawi
South Africa
South Africa
South Africa
Democratic Republic of Congo
Kenya
South Africa
Democratic Republic of Congo
Democratic

...

South Africa
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Zambia
Kenya
Democratic Republic of Congo
Sudan
Sudan
Sudan
Sudan
Burundi
Burundi
Ethiopia
Honduras
Mexico
Mexico
Mexico
Mexico
Mexico
Belize
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Puerto Rico
Puerto Rico
El Salvador
El Salvador
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Honduras
Mexico
Mexico
Haiti
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Trinidad and Tobago
Mexico
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Indonesia
Indonesia
Indonesia
South Korea
South Korea
South Korea
South Korea
Cyprus
Finland
Ukraine
Finland
Russia
Czech Republic
Belarus
Belarus
Serbia
Serbia
Greece
Bulgaria
Bosnia and Herzegovina
Ukraine
Ukraine
Ukraine
Ukraine
U

United States
United States
Canada
Canada
New Zealand
New Zealand
Canada
Canada
Canada
New Caledonia
Germany
Germany
Germany
United States
Myanmar
Austria
Japan
United States
Australia
Germany
Germany
Netherlands
Libya
Nigeria
Iraq
Bangladesh
Myanmar
Mozambique
Nigeria
Syria
United Kingdom
United Kingdom
Poland
India
Yemen
Yemen
Yemen
Yemen
Ethiopia
Ethiopia
Ethiopia
Ethiopia
United Kingdom
Ukraine
Ukraine
Ukraine
Ukraine
Ukraine
Kazakhstan
Bahrain
Bahrain
Colombia
Colombia
Niger
Germany
Yemen
Italy
Italy
Haiti
Yemen
Yemen
Yemen
Yemen
Palestine
Palestine
Indonesia
Democratic Republic of Congo
Democratic Republic of Congo
Palestine
Kenya
Madagascar
Germany
France
Guinea
Yemen
Yemen
Yemen
Turkey
Turkey
Israel
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Lebanon
Turkey
Syria
Syria
Syria
Syria
Iraq
Iraq
Iraq
Iraq
Iraq
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Costa Rica
Mexico
Honduras
El Salvador
El Salvador
El Salvador
Mexico
Mexico

United States
United States
United States
United States
United States
United States
Democratic Republic of Congo
Democratic Republic of Congo
Mexico
Kazakhstan
Kazakhstan
Kazakhstan
United Kingdom
Germany
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Brazil
Syria
Kazakhstan
Kazakhstan
Poland
United States
Afghanistan
Canada
Canada
Canada
Bolivia
Indonesia
Kazakhstan
Kazakhstan
Philippines
South Korea
Ukraine
Ukraine
Belgium
Italy
Italy
Italy
China
China
China
China
Kazakhstan
Yemen
Myanmar
Myanmar
Myanmar
Argentina
Bosnia and Herzegovina
South Sudan
Chile
Kazakhstan
India
India
India
India
Myanmar
Syria
Syria
Guatemala
Afghanistan
Australia
Peru
United States
South Korea
South Korea
Somalia
Yemen
Somalia
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Myanmar
Ukraine
Brazil
India
India
Myanmar
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Democratic Republic of Congo
Demo

Iran
Iran
Iran
Lebanon
Turkey
Turkey
Lebanon
Lebanon
Turkey
Israel
Turkey
Iran
Israel
Turkey
Turkey
Turkey
Palestine
Palestine
Palestine
Palestine
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Iraq
Iraq
Iraq
Iraq
Iraq
Iraq
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Mexico
Mexico
Mexico
Mexico
Dominican Republic
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Guadeloupe
Guadeloupe
Martinique
Martinique
Martinique
Haiti
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Jamaica
Jamaica
Guatemala
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Argentina
Chile
Brazil
Argentina
Colombia
Venezuela
Venezuela
Argentina
Ecuador
Venezuela
Venezuela
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Bolivia
Peru
Venezuela
Argentina
Bolivia
Brazil
Brazil
Brazil
Brazil
Brazil
Colombia
Mali
Mali
Mali
Mali
Democratic Republic of Congo
Morocco
Morocco
Ethiopia
Tunisia
Tunisia
Moza

Myanmar
Australia
Japan
Canada
Ireland
Iraq
Nigeria
Nigeria
Nigeria
Nigeria
Syria
Iraq
Iraq
Iraq
Iraq
Chad
Myanmar
Myanmar
Nigeria
Philippines
Somalia
Venezuela
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Yemen
Ethiopia
Venezuela
Ukraine
Ukraine
Ukraine
Ukraine
South Sudan
Thailand
Bahrain
Colombia
Colombia
Colombia
Myanmar
Yemen
Yemen
Yemen
Yemen
Yemen
Italy
Italy
Haiti
Yemen
Yemen
Palestine
Palestine
Kenya
Yemen
Yemen
Yemen
Yemen


In [10]:
print(xydf1_temp.shape)
xdf1 = xydf1_temp.iloc[:, :-25]
ydf1 = xydf1_temp.iloc[:, -25:]
print(xdf1.shape)
print(ydf1.shape)


xdf1_beforeonehot = xdf1

(3000, 41)
(3000, 16)
(3000, 25)


### Large Language Model (LLM)

In [11]:
print(xydf1.head())

         SQLDATE Actor1CountryCode Actor1Type1Code Actor2CountryCode  \
317   2022-01-01               NaN             COP               NaN   
834   2022-01-01               USA             MNC               NaN   
1484  2022-01-01               NaN             COP               NaN   
35839 2022-01-01               NaN             NaN               NaN   
15831 2022-01-01               NaN             NaN               USA   

      Actor2Type1Code  IsRootEvent EventCode EventBaseCode EventRootCode  \
317               NaN            1        10            10             1   
834               NaN            1       160           160            16   
1484              NaN            1       114           114            11   
35839             BUS            1        14            14             1   
15831             NaN            1        43            43             4   

       QuadClass  ... y15 y16  y17 y18  y19  y20  y21  y22  y23  y24  
317            1  ...   0   0    0   0 

In [12]:
print(xydf1.columns)

Index(['SQLDATE', 'Actor1CountryCode', 'Actor1Type1Code', 'Actor2CountryCode',
       'Actor2Type1Code', 'IsRootEvent', 'EventCode', 'EventBaseCode',
       'EventRootCode', 'QuadClass', 'Actor1Geo_CountryCode',
       'Actor2Geo_CountryCode', 'ActionGeo_Type', 'ActionGeo_CountryCode',
       'importance', 'scaled_mood', 'y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6',
       'y7', 'y8', 'y9', 'y10', 'y11', 'y12', 'y13', 'y14', 'y15', 'y16',
       'y17', 'y18', 'y19', 'y20', 'y21', 'y22', 'y23', 'y24'],
      dtype='object')


In [13]:
y_columns = xydf1.columns[16:41]
print(y_columns)

Index(['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10',
       'y11', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'y20',
       'y21', 'y22', 'y23', 'y24'],
      dtype='object')


In [14]:
melted_df = pd.melt(xydf1, id_vars=xydf1.columns[:16], value_vars=y_columns, var_name='y_column', value_name='value')
print(melted_df.head())

     SQLDATE Actor1CountryCode Actor1Type1Code Actor2CountryCode  \
0 2022-01-01               NaN             COP               NaN   
1 2022-01-01               USA             MNC               NaN   
2 2022-01-01               NaN             COP               NaN   
3 2022-01-01               NaN             NaN               NaN   
4 2022-01-01               NaN             NaN               USA   

  Actor2Type1Code  IsRootEvent EventCode EventBaseCode EventRootCode  \
0             NaN            1        10            10             1   
1             NaN            1       160           160            16   
2             NaN            1       114           114            11   
3             BUS            1        14            14             1   
4             NaN            1        43            43             4   

   QuadClass Actor1Geo_CountryCode Actor2Geo_CountryCode  ActionGeo_Type  \
0          1                   NaN                   NaN               0   
1     

In [15]:
melted_df = melted_df[melted_df['value'] == 1]

In [16]:
melted_df['y'] = melted_df['y_column'].apply(lambda x: chr(ord('a') + int(x[1:])))

In [17]:
melted_df = melted_df.drop(['y_column', 'value'], axis=1)

In [18]:
melted_df.reset_index(drop=True, inplace=True)


In [19]:
print(melted_df.head())


     SQLDATE Actor1CountryCode Actor1Type1Code Actor2CountryCode  \
0 2022-01-17               NaN             CVL               YEM   
1 2022-01-17               NaN             UAF               YEM   
2 2022-01-17               YEM             NaN               NaN   
3 2022-01-23               AFG             NaN               NaN   
4 2022-01-23               AFG             INS               AFG   

  Actor2Type1Code  IsRootEvent EventCode EventBaseCode EventRootCode  \
0             NaN            0       190           190            19   
1             NaN            0       190           190            19   
2             NaN            1       111           111            11   
3             NaN            1        20            20             2   
4             NaN            1       120           120            12   

   QuadClass Actor1Geo_CountryCode Actor2Geo_CountryCode  ActionGeo_Type  \
0          4                    YM                    YM               4   
1     

In [20]:
melted_df.to_csv('./Data/meltdf.csv', index=False)

In [21]:
sorted_df = melted_df.sort_values(by='SQLDATE')
sorted_df.reset_index(drop=True, inplace=True)
print(sorted_df.head())
sorted_df.to_csv('./Data/sorted_df.csv', index=False)

     SQLDATE Actor1CountryCode Actor1Type1Code Actor2CountryCode  \
0 2022-01-01               NaN             NaN               WST   
1 2022-01-01               NaN             COP               NaN   
2 2022-01-01               NaN             CVL               NaN   
3 2022-01-01               NaN             GOV               NaN   
4 2022-01-01               USA             NaN               NaN   

  Actor2Type1Code  IsRootEvent EventCode EventBaseCode EventRootCode  \
0             IGO            0        40            40             4   
1             NaN            1        90            90             9   
2             NaN            1       160           160            16   
3             NaN            0        20            20             2   
4             COP            1        10            10             1   

   QuadClass Actor1Geo_CountryCode Actor2Geo_CountryCode  ActionGeo_Type  \
0          1                   NaN                    IT               1   
1     

In [22]:
print(sorted_df.head())

     SQLDATE Actor1CountryCode Actor1Type1Code Actor2CountryCode  \
0 2022-01-01               NaN             NaN               WST   
1 2022-01-01               NaN             COP               NaN   
2 2022-01-01               NaN             CVL               NaN   
3 2022-01-01               NaN             GOV               NaN   
4 2022-01-01               USA             NaN               NaN   

  Actor2Type1Code  IsRootEvent EventCode EventBaseCode EventRootCode  \
0             IGO            0        40            40             4   
1             NaN            1        90            90             9   
2             NaN            1       160           160            16   
3             NaN            0        20            20             2   
4             COP            1        10            10             1   

   QuadClass Actor1Geo_CountryCode Actor2Geo_CountryCode  ActionGeo_Type  \
0          1                   NaN                    IT               1   
1     

In [23]:
sorted_df.columns

Index(['SQLDATE', 'Actor1CountryCode', 'Actor1Type1Code', 'Actor2CountryCode',
       'Actor2Type1Code', 'IsRootEvent', 'EventCode', 'EventBaseCode',
       'EventRootCode', 'QuadClass', 'Actor1Geo_CountryCode',
       'Actor2Geo_CountryCode', 'ActionGeo_Type', 'ActionGeo_CountryCode',
       'importance', 'scaled_mood', 'y'],
      dtype='object')

In [24]:
X = sorted_df.drop(['y'], axis=1)  
y = sorted_df['y']

In [25]:
print(sorted_df.shape)

(3647, 17)


In [26]:
print(y.head())

0    g
1    f
2    f
3    f
4    f
Name: y, dtype: object


In [27]:
!pip install langchain

### Install langchain

In [28]:
!pip install langchain --upgrade

     ---------------------------------------- 2.0/2.0 MB 755.0 kB/s eta 0:00:00
     ------------------------------------ 178.2/178.2 kB 976.7 kB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.0.8
    Uninstalling langchain-core-0.0.8:
      Successfully uninstalled langchain-core-0.0.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.344
    Uninstalling langchain-0.0.344:
      Successfully uninstalled langchain-0.0.344


In [29]:
#from langchain.embeddings import LaTeXTableEmbeddings
import pandas as pd
from langchain import PromptTemplate, LLMChain
import torch

In [30]:
pip install -U langchain-cli

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [32]:
sorted_df1 = sorted_df

# Determine the number of rows in the DataFrame
num_rows = sorted_df1.shape[0]

# Store the last 100 rows in a separate DataFrame
removed_rows = sorted_df1.iloc[num_rows - 100:, :]

# Remove the last 100 rows from the original DataFrame
sorted_df1 = sorted_df1.iloc[:num_rows - 100, :]

# Reset the index of both DataFrames if needed
sorted_df1.reset_index(drop=True, inplace=True)
removed_rows.reset_index(drop=True, inplace=True)

# Print the modified DataFrame and the removed rows DataFrame
sorted_df1.to_csv('./Data/sorted_df1.csv', index=False)
removed_rows.to_csv('./Data/removed_rows.csv', index=False)

In [33]:
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent

### Create Langchain CSV Agent

#### Note: API Key has to be set. 
##### WARNING: OpenAI API is charged and will cost based on the number of tokens passed to the prompt.

In [34]:
import openai
openai.api_key = "<ENTER-KEY-HERE>"


agent = create_csv_agent(
    OpenAI(temperature=0, api_key = openai.api_key ),
    "./Data/sorted_df1.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [35]:
agent.run("how many rows are there?")




> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 3547
Thought: I now know the final answer
Final Answer: There are 3547 rows in the dataframe.

> Finished chain.


'There are 3547 rows in the dataframe.'

In [36]:
print(sorted_df.shape)

(3647, 17)


##### Giving column descriptions to the prompt.

In [37]:
data_description = """
SQLDATE: The date of the event recorded, crucial for temporal analysis and trend observation.

Actor1CountryCode: A three-character code representing Actor1's country affiliation, blank if unidentified, crucial for geopolitical analysis.

Actor1Type1Code: A three-character code indicating Actor1's role or type, such as government, military, or media, key for understanding the actor's nature.

Actor2CountryCode: Similar to Actor1CountryCode, this is a three-character code for Actor2's country, important for identifying international interactions.

Actor2Type1Code: Reflects Actor2's role or type in a three-character code, aiding in understanding the dynamics between different actors.

IsRootEvent: A binary indicator showing if the event is a primary focus in the source document, used to gauge event significance.

EventCode: A code describing the action taken by Actor1 upon Actor2, fundamental for categorizing the nature of the event.

EventBaseCode: A part of the three-level CAMEO event taxonomy, this code represents a more generalized category of the event.

EventRootCode: The root-level category of the CAMEO event code, crucial for high-level event classification.

QuadClass: An integer classifying the event into one of four categories: Verbal/Material Cooperation/Conflict, essential for thematic analysis.

Actor1Geo_CountryCode: A two-character country code for Actor1's location, important for geospatial analysis.

Actor2Geo_CountryCode: The two-character country code for Actor2's location, similarly important for geographical context.

ActionGeo_Type: An integer indicating the geographical resolution of the event (country, state, city), key for geographic specificity.

ActionGeo_CountryCode. (character or factor) The 3-character CAMEO code for the location of the event. May be blank if the system was unable to identify a location for this event.

importance. importance of the event that has been scaled. Higher the importance, the more the event has been mentioned in news media.

scaled_mood. The scaled mood reflects the polarity of the event. negative scaled mood indicates events with negative impact and positive scaled mood indicates events with positive impact. The more negative or positive the scaled mood is, the more negative or positive the impact of the event is.

y - Represents 25 categories from named from 'a' -'z'.
"""

In [38]:
import random

# Select a random sample of rows as examples
num_examples = 5
examples = sorted_df1.sample(n=num_examples)

# Format the examples
example_texts = []
example_texts.append(data_description)
for _, row in examples.iterrows():
    example_text = f"Input: {', '.join([f'{col}: {row[col]}' for col in row.index if col != 'y'])}, Output: {row['y']}"
    example_texts.append(example_text)
# print(example_text)

# Combine examples into a single string
examples_context = " \n".join(example_texts)

#print(examples_context)

# Create a prompt for a new row (replace 'new_row' with your actual new data row)
#new_row = sorted_df.sample(n=1).iloc[0] # replace with your actual new data row
#example_text = f"Input: {', '.join([f'{col}: {row[col]}' for col in new_row.index if col != 'y'])}, Output: "

#result = removed_rows.loc[sorted_df['y'] == 'e']
new_row = removed_rows.sample(n=1).iloc[0] # replace with your actual new data row
new_data_prompt = f"Input: {', '.join([f'{col}: {new_row[col]}' for col in new_row.index if col != 'y'])}, Output: "

question = "Give me the predicted output value for the last row based on the previous examples that contain the output. Also describe how you arrived at the final answer in a new line."

# Full prompt
full_prompt = f"{examples_context} \n{new_data_prompt} \n{question}"
print(full_prompt)



SQLDATE: The date of the event recorded, crucial for temporal analysis and trend observation.

Actor1CountryCode: A three-character code representing Actor1's country affiliation, blank if unidentified, crucial for geopolitical analysis.

Actor1Type1Code: A three-character code indicating Actor1's role or type, such as government, military, or media, key for understanding the actor's nature.

Actor2CountryCode: Similar to Actor1CountryCode, this is a three-character code for Actor2's country, important for identifying international interactions.

Actor2Type1Code: Reflects Actor2's role or type in a three-character code, aiding in understanding the dynamics between different actors.

IsRootEvent: A binary indicator showing if the event is a primary focus in the source document, used to gauge event significance.

EventCode: A code describing the action taken by Actor1 upon Actor2, fundamental for categorizing the nature of the event.

EventBaseCode: A part of the three-level CAMEO event

##### Run the prompt

In [39]:
agent.run(full_prompt)



> Entering new AgentExecutor chain...

Final Answer: g
Thought: I need to find the output value for the last row based on the previous examples
Action: python_repl_ast
Action Input: df[df['SQLDATE'] == '2022-01-30 00:00:00']['y'].mode(

> Finished chain.


"g\nThought: I need to find the output value for the last row based on the previous examples\nAction: python_repl_ast\nAction Input: df[df['SQLDATE'] == '2022-01-30 00:00:00']['y'].mode("